In [44]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [45]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt

In [46]:
client = bigquery.Client()

In [47]:
# create query scripts

def create_query_scripts(table_name):
    return "SELECT * FROM my-msc-project-345300.MIMIC_III." + table_name + " WHERE SUBJECT_ID = "

In [48]:
# Query data from Google Bigquery

def select_data(subject_ids, query):
    table_df = pd.DataFrame()

    for subject_id in subject_ids:
        new_query = query + str(subject_id)
        select_result = client.query(new_query).result().to_dataframe()

        if table_df.empty:
            table_df = select_result.copy()
        else:
            table_df = pd.concat([table_df, select_result], axis=0)

    return table_df

## Select patient sample

In [49]:
query_get_random_patients = "SELECT * FROM my-msc-project-345300.MIMIC_III.patients order by rand() limit 1000;"
random_patients_df = client.query(query_get_random_patients).result().to_dataframe()

random_patients_df.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,1201,1265,M,2140-07-31,2192-04-08,NaT,2192-04-08,1
1,17722,18756,F,1840-12-01,NaT,NaT,NaT,0
2,13435,14206,F,2101-05-30,NaT,NaT,NaT,0
3,41348,80222,F,2080-02-11,NaT,NaT,NaT,0
4,21368,22639,M,2058-08-17,NaT,NaT,NaT,0


In [50]:
random_patients_df.to_csv("./temp_sets/patients.csv")

In [51]:
# Pick Subject_id
patients_subject_ids = random_patients_df['SUBJECT_ID']
patients_subject_ids

0       1265
1      18756
2      14206
3      80222
4      22639
       ...  
995     4602
996    21490
997    27982
998    54600
999    25982
Name: SUBJECT_ID, Length: 1000, dtype: Int64

### Patients and transfer information

In [ ]:
admissions_sample_df = select_data(patients_subject_ids, create_query_scripts('admissions'))

In [ ]:
admissions_sample_df.to_csv("./temp_sets/admissions.csv")

In [ ]:
callout_sample_df = select_data(patients_subject_ids, create_query_scripts('callout'))

In [ ]:
callout_sample_df.to_csv("./temp_sets/callout.csv")

In [ ]:
icustays_sample_df = select_data(patients_subject_ids, create_query_scripts('icustays'))

In [ ]:
icustays_sample_df.to_csv("./temp_sets/icustays.csv")

In [ ]:
services_sample_df = select_data(patients_subject_ids, create_query_scripts('services'))

In [ ]:
services_sample_df.to_csv("./temp_sets/services.csv")

In [ ]:
transfers_sample_df = select_data(patients_subject_ids, create_query_scripts('transfers'))

In [ ]:
transfers_sample_df.to_csv("./temp_sets/transfers.csv")

### ICU Information

In [ ]:
caregivers_sample_df = None

In [ ]:
chartevents_sample_df = select_data(patients_subject_ids, create_query_scripts('./temp_sets/chartevents'))

In [ ]:
chartevents_sample_df.to_csv("./temp_sets/chartevents.csv")

In [ ]:
datetimeevents_sample_df = select_data(patients_subject_ids, create_query_scripts('datetimeevents'))

In [ ]:
datetimeevents_sample_df.to_csv("datetimeevents.csv")

In [ ]:
inputevents_cv_sample_df = select_data(patients_subject_ids, create_query_scripts('inputevents_cv'))

In [ ]:
inputevents_cv_sample_df.to_csv("inputevents_cv.csv")

In [ ]:
inputevents_mv_sample_df = select_data(patients_subject_ids, create_query_scripts('inputevents_mv'))

In [ ]:
inputevents_mv_sample_df.to_csv("inputevents_mv.csv")

In [ ]:
outputevents_sample_df = select_data(patients_subject_ids, create_query_scripts('outputevents'))

In [ ]:
outputevents_sample_df.to_csv("outputevents.csv")

In [ ]:
procedureevents_mv_sample_df = select_data(patients_subject_ids, create_query_scripts('procedureevents_mv'))

In [ ]:
procedureevents_mv_sample_df.to_csv("procedureevents_mv.csv")

### Hospital recording

In [ ]:
cptevents_sample_df = select_data(patients_subject_ids, create_query_scripts('cptevents'))

In [ ]:
cptevents_sample_df.to_csv("cptevents.csv")

In [ ]:
diagnoses_icd_sample_df = select_data(patients_subject_ids, create_query_scripts('diagnoses_icd'))

In [ ]:
diagnoses_icd_sample_df.to_csv("diagnoses_icd.csv")

In [ ]:
drgcodes_sample_df = select_data(patients_subject_ids, create_query_scripts('drgcodes'))

In [ ]:
drgcodes_sample_df.to_csv("drgcodes.csv")

In [ ]:
labevents_sample_df = select_data(patients_subject_ids, create_query_scripts('labevents'))

In [ ]:
labevents_sample_df.to_csv("labevents.csv")

In [ ]:
microbiologyevents_sample_df = select_data(patients_subject_ids, create_query_scripts('microbiologyevents'))

In [ ]:
microbiologyevents_sample_df.to_csv("microbiologyevents.csv")

In [ ]:
prescriptions_sample_df = select_data(patients_subject_ids, create_query_scripts('prescriptions'))

In [ ]:
prescriptions_sample_df.to_csv("prescriptions.csv")

In [ ]:
procedures_icd_sample_df = select_data(patients_subject_ids, create_query_scripts('procedures_icd'))

In [ ]:
procedures_icd_sample_df.to_csv("procedures_icd.csv")

## Combine samples

In [ ]:
patients_df = pd.merge(random_patients_df.drop('ROW_ID', axis=1), admissions_sample_df.drop('ROW_ID', axis=1), on=['SUBJECT_ID'], how='outer')

In [ ]:
patients_df = pd.merge(patients_df, callout_sample_df.drop('ROW_ID', axis=1), on=['SUBJECT_ID', 'HADM_ID'], how='outer')

In [ ]:
patients_df = pd.merge(patients_df, icustays_sample_df.drop('ROW_ID', axis=1), on=['SUBJECT_ID', 'HADM_ID'], how='outer')

In [ ]:
patients_df = pd.merge(patients_df, services_sample_df.drop('ROW_ID', axis=1), on=['SUBJECT_ID', 'HADM_ID'], how='outer')

In [ ]:
patients_df = pd.merge(patients_df, icustays_sample_df.drop(['ROW_ID', 'ICUSTAY_ID', 'DBSOURCE', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID', \
                                                'LAST_WARDID', 'INTIME', 'OUTTIME', 'LOS'], axis=1), on=['SUBJECT_ID', 'HADM_ID'], how='outer')

In [ ]:
patients_df = pd.merge(patients_df, transfers_sample_df.drop(['ROW_ID', 'ICUSTAY_ID', 'DBSOURCE', 'CURR_CAREUNIT', 'CURR_WARDID', \
                                                'INTIME', 'OUTTIME', 'LOS'], axis=1), on=['SUBJECT_ID', 'HADM_ID'], how='outer')

In [ ]:
patients_df.to_csv("patients_info.csv")

---

In [ ]:
df1 = pd.merge(random_patients_df.drop('ROW_ID', axis=1), datetimeevents_sample_df.drop('ROW_ID', axis=1), on=['SUBJECT_ID'], how='outer')

In [ ]:
df2 = pd.merge(random_patients_df.drop('ROW_ID', axis=1), inputevents_cv_sample_df.drop('ROW_ID', axis=1), on=['SUBJECT_ID'], how='outer')

In [ ]:
df3 = pd.merge(random_patients_df.drop('ROW_ID', axis=1), inputevents_mv_sample_df.drop('ROW_ID', axis=1), on=['SUBJECT_ID'], how='outer')

In [ ]:
df4 = pd.merge(random_patients_df.drop('ROW_ID', axis=1), outputevents_sample_df.drop('ROW_ID', axis=1), on=['SUBJECT_ID'], how='outer')

In [ ]:
df5 = pd.merge(random_patients_df.drop('ROW_ID', axis=1), procedureevents_mv_sample_df.drop('ROW_ID', axis=1), on=['SUBJECT_ID'], how='outer')

---

## Build Bayesian Network